# Freshman On-Track (FOT) Intervention Recommender
### A Standalone Proof-of-Concept

**Goal:** To show, in a few simple steps, how we can turn a description of a struggling student into a set of clear, actionable, and evidence-based recommendations for an educator.

This notebook demonstrates the core Retrieval-Augmented Generation (RAG) pipeline that powers our recommender.

## Step 1: Setting Up the Environment

First, we need to load the project's code and install its dependencies. This cell prepares the notebook to run our custom logic.

*(This notebook is designed to run in Google Colab, but the code below will also adapt to a local environment if the project files are present.)*

In [8]:
import sys, os, warnings
from pathlib import Path
from tqdm import TqdmWarning

# This prevents common, harmless warnings from cluttering the output.
os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings("ignore", category=TqdmWarning)

# Clones the project from GitHub, but only if it doesn't already exist.
PROJECT_DIR = "fot-intervention-recommender"
if not Path(PROJECT_DIR).is_dir():
    print("🚀 Downloading project files...")
    !git clone -q https://github.com/chuckfinca/fot-intervention-recommender.git

# Installs packages and adds the project's code to our Python path.
print("📦 Setting up the environment...")
!{sys.executable} -m pip install -q -r {PROJECT_DIR}/requirements.txt
sys.path.insert(0, str(Path(PROJECT_DIR) / "src"))

# Define the project_path variable needed by the rest of the notebook
project_path = Path(PROJECT_DIR)

print("✅ Environment is ready!")

📦 Setting up the environment...
/Users/charlesfeinn/Developer/job_applications/fot-intervention-recommender/.venv/bin/python3: No module named pip
✅ Environment is ready!


## Step 2: Define the Student (The Input)

Everything starts with a student. Our system takes a simple narrative summary that an educator might write. This summary describes the student's challenges in plain English. 

Let's use the sample profile from the project description.

In [2]:
from IPython.display import display, Markdown

student_profile = {
    "narrative_summary": "This student is struggling to keep up with coursework, "
    "having failed one core class and earning only 2.5 credits out of 4 credits "
    "expected for the semester. Attendance is becoming a concern at 88% for an average "
    "annual target of 90%, and they have had one behavioral incident. "
    "The student needs targeted academic and attendance support to get back on track for graduation."
}

student_query = student_profile["narrative_summary"]

display(Markdown(f"**Student Query:**\n> {student_query}"))

**Student Query:**
> This student is struggling to keep up with coursework, having failed one core class and earning only 2.5 credits out of 4 credits expected for the semester. Attendance is becoming a concern at 88% for an average annual target of 90%, and they have had one behavioral incident. The student needs targeted academic and attendance support to get back on track for graduation.

## Step 3: Find Relevant Strategies (The "Retrieval" Step)

Now, we take the student's story and find the most relevant strategies from our **Knowledge Base**—a curated library of best practices and proven interventions.

How do we do this? 
1.  We've already converted our knowledge base documents into **vector embeddings** (unique digital fingerprints that capture meaning).
2.  We use a **FAISS vector database**—a super-fast search index—to instantly find the documents with fingerprints most similar to the student's situation.

Let's see which top 3 strategies our system retrieves for this student.

In [3]:
# Import the necessary functions from our project's code
from fot_recommender.rag_pipeline import (
    load_knowledge_base,
    initialize_embedding_model,
    create_embeddings,
    create_vector_db,
    search_interventions,
    generate_recommendation_summary
)
from fot_recommender.utils import display_recommendations

# --- Load all the components of our RAG system ---

# 1. Load the chunked knowledge base
kb_path = project_path / "data" / "processed" / "knowledge_base_final_chunks.json"
knowledge_base_chunks = load_knowledge_base(str(kb_path))

# 2. Initialize the embedding model
embedding_model = initialize_embedding_model()

# 3. Create embeddings and the vector database
embeddings = create_embeddings(knowledge_base_chunks, embedding_model)
vector_db = create_vector_db(embeddings)

# --- Perform the search! ---
retrieved_interventions = search_interventions(
    query=student_query,
    model=embedding_model,
    index=vector_db,
    knowledge_base=knowledge_base_chunks,
    k=3,
    min_similarity_score=0.4
)

# Display the titles of what we found
display(Markdown("**Top 3 Retrieved Strategies:**"))
for chunk, score in retrieved_interventions:
    display(Markdown(f"- **{chunk['title']}** (Source: *{chunk['source_document']}*, Relevance: {score:.2f})"))

/Users/charlesfeinn/Developer/job_applications/fot-intervention-recommender/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Initializing embedding model: all-MiniLM-L6-v2...
Model initialized successfully.
Creating embeddings for 27 chunks...


Batches:   0%|                                            | 0/1 [00:00<?, ?it/s]/Users/charlesfeinn/Developer/job_applications/fot-intervention-recommender/.venv/lib/python3.12/site-packages/torch/nn/modules/module.py:1520: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Batches: 100%|████████████████████████████████████| 1/1 [00:02<00:00,  2.08s/it]


Embeddings created successfully.
Creating FAISS index with dimension 384...
FAISS index created with 27 vectors.

Searching for top 3 interventions for query: 'This student is struggling to keep up with coursework, having failed one core cl...'
Found 3 relevant interventions.


**Top 3 Retrieved Strategies:**

- **Strategy: Differentiating Intervention Tiers** (Source: *NCS_OTToolkit_2ndEd_October_2017_updated.pdf*, Relevance: 0.57)

- **Tool: Intervention Tracking** (Source: *NCS_OTToolkit_2ndEd_October_2017_updated.pdf*, Relevance: 0.54)

- **Tool: BAG Report (Example)** (Source: *NCS_OTToolkit_2ndEd_October_2017_updated.pdf*, Relevance: 0.53)

## Step 4: Create the Recommendation (The "Generation" Step)

Finding the right documents is only half the battle. Raw research isn't very helpful to a busy teacher. 

In this final step, we use a powerful Large Language Model (Google's Gemini API) to act as an expert instructional coach. We give it the student's story and the relevant strategies we just retrieved. The AI's job is to **synthesize** this information into a concise, practical recommendation tailored specifically for a teacher.

This is the final output of our system.

In [4]:
from dotenv import load_dotenv

# Load the API key from a .env file (if it exists)
load_dotenv(project_path / '.env') 
api_key = os.getenv("FOT_GOOGLE_API_KEY")

if not api_key:
    print("✋ FOT_GOOGLE_API_KEY not found. Please provide your Google API key to generate the summary.")
    final_recommendation = "(API Key not provided - could not generate summary)"
else:
    final_recommendation = generate_recommendation_summary(
        retrieved_chunks=retrieved_interventions,
        student_narrative=student_query,
        api_key=api_key,
        persona="teacher"
    )

display(Markdown("### Final Synthesized Recommendation for the Teacher"))
display(Markdown(final_recommendation))


Synthesizing recommendation for persona: 'teacher' using Gemini...
Synthesis complete.


### Final Synthesized Recommendation for the Teacher

This student is experiencing academic difficulty, reflected in a 2.5 GPA and a failing grade in one core class, coupled with attendance concerns (88% attendance versus a 90% target) and one behavioral incident.  To address these challenges and support the student's path to graduation, the following interventions are recommended:


**1. Implement a Tiered Intervention Strategy:**  Determine the extent to which attendance is contributing to the student's academic struggles. ("Strategy: Differentiating Intervention Tiers").  If attendance is a significant factor,  refer the student to the appropriate support services (Success Team or Attendance Dean, as indicated by the BAG Report format) to address these issues directly. This allows more focused support from the teaching staff for academic interventions.

**2. Utilize a Robust Intervention Tracking System:**  Implement a system to monitor the student's progress, focusing on attendance, GPA, and behavior. ("Tool: Intervention Tracking").  This system should clearly document interventions (e.g., tutoring sessions, mentorship meetings), and track the student’s progress in each core course (GPA and attendance rates) at two checkpoints within a ten-week period. The "BAG Report" format provides a useful template to track behavior, attendance and grades. This data will inform adjustments to the support plan.

**3.  Regularly Review the Student's "BAG Report" (or Equivalent):**  Use a reporting mechanism (such as the BAG report example) to regularly review the student's performance across all three key areas: Behavior, Attendance, and Grades. This visual representation highlights areas of strength and areas requiring immediate intervention, allowing for proactive adjustments to support strategies. This aligns with the recommendation to monitor multiple key performance indicators to improve student outcomes effectively.


## Bonus: See the Evidence

The recommendation above isn't just made up—it's directly grounded in the documents we retrieved. Here are the specific text snippets that the AI used to create its summary. This ensures our recommendations are always transparent and evidence-based.

In [5]:
display_recommendations(retrieved_interventions)


--- Top Recommended Interventions ---

--- Recommendation 1 (Similarity Score: 0.5735) ---
  Title: Strategy: Differentiating Intervention Tiers
  Source: NCS_OTToolkit_2ndEd_October_2017_updated.pdf (Pages: 46)
  
  Content Snippet:
  "To what degree is attendance playing a role in student performance? To whom do you refer Tier 3 students who have serious attendance issues (inside and outside of the school) so that the Success Team can really concentrate on supporting Tier 2 students?..."
--------------------------------------------------

--- Recommendation 2 (Similarity Score: 0.5416) ---
  Title: Tool: Intervention Tracking
  Source: NCS_OTToolkit_2ndEd_October_2017_updated.pdf (Pages: 49)
  
  Content Snippet:
  "Features of Good Intervention Tracking Tools:
  • Name of the intervention and what key performance indicator it addresses (attendance, point-in-time On-Track rates, GPA, behavior metric, etc.)
  • Names of the targeted students
    ° If tracking grades, include each cor

## Explore the Live Demo!

You've seen the step-by-step process of how our RAG system turns a student's story into an actionable, evidence-based plan. Now, it's time to try it yourself with any student scenario you can imagine!

We have deployed this entire system as an interactive web application on Hugging Face Spaces. Click the link below to access the live demo—no setup or API key required.


#### [👉 Click Here to Launch the Live FOT Recommender API](https://huggingface.co/spaces/chuckfinca/fot-recommender-api)
